# Libraries and create tiles

In [2]:
import stlearn as st
st.settings.set_figure_params(dpi=300)
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import sys; import scanpy as sc
from pathlib import Path
from typing import Optional, Union
from anndata import AnnData
import pandas as pd
import stlearn
from typing import Optional, Union
from anndata import AnnData
from pathlib import Path
from PIL import Image
from tqdm import tqdm
import seaborn as sns
import numpy as np
import os
from sklearn.multioutput import MultiOutputClassifier
import pandas as pd
import lightgbm as lgb
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import LabelBinarizer
import os
from keras.applications.imagenet_utils import decode_predictions
from keras.preprocessing import image
from sklearn.metrics import roc_auc_score
from skimage.color import rgb2hed
%matplotlib inline
import pandas as pd
from keras.utils import to_categorical
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import sys
from numpy import load
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras import backend
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout, BatchNormalization
from keras.optimizers import SGD, Adam
from keras.models import Model
from tensorflow.keras import regularizers
from keras.applications.resnet50 import ResNet50, preprocess_input
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import lime
from sklearn.preprocessing import MinMaxScaler
from lime import lime_image
from skimage.segmentation import mark_boundaries
from skimage.segmentation import watershed
import glob
import os
from tensorflow.keras.preprocessing import image as image_fun
from sklearn.preprocessing import OneHotEncoder
import skimage
from skimage.color import rgb2hed
from skimage.feature import peak_local_max
from skimage.segmentation import watershed
from skimage.measure import label
import scipy as sp
from scipy import ndimage as ndi
from skimage.morphology import area_opening
import math; import copy


def tiling(
        adata: AnnData,
        out_path: Union[Path, str] = "./tiling",
        library_id: str = None,
        crop_size: int = 40,
        target_size: int = 299,
        verbose: bool = False,
        copy: bool = False,
) -> Optional[AnnData]:
    

    if library_id is None:
        library_id = list(adata.uns["spatial"].keys())[0]

    # Check the exist of out_path
    if not os.path.isdir(out_path):
        os.mkdir(out_path)

    image = adata.uns["spatial"][library_id]["images"][adata.uns["spatial"]["use_quality"]]
    if image.dtype == np.float32 or image.dtype == np.float64:
        image = (image * 255).astype(np.uint8)
    img_pillow = Image.fromarray(image)
    tile_names = []

    with tqdm(
            total=len(adata),
            desc="Tiling image",
            bar_format="{l_bar}{bar} [ time left: {remaining} ]",
    ) as pbar:
        for imagerow, imagecol in zip(adata.obs["imagerow"], adata.obs["imagecol"]):
            imagerow_down = imagerow - crop_size / 2
            imagerow_up = imagerow + crop_size / 2
            imagecol_left = imagecol - crop_size / 2
            imagecol_right = imagecol + crop_size / 2
            tile = img_pillow.crop(
                (imagecol_left, imagerow_down, imagecol_right, imagerow_up)
            )
            # tile.thumbnail((target_size, target_size), Image.ANTIALIAS)
            tile = tile.resize((target_size, target_size))
            tile_name = library_id + "-" + str(imagecol) + "-" + str(imagerow) + "-" + str(crop_size)#np.arange(len(pd.Series(adata))+1).astype(str).str.zfill(4)+1 + "-" +
            out_tile = Path(out_path) / (tile_name + ".jpeg")
            tile_names.append(str(out_tile))
            if verbose:
                print(
                    "generate tile at location ({}, {})".format(
                        str(imagecol), str(imagerow)
                    )
                )
            tile.save(out_tile, "JPEG")

            pbar.update(1)

    adata.obs["tile_path"] = tile_names
    return adata if copy else None

# ST-Net train set

In [8]:
BASE_PATH = Path("D:/onkar/Projects/Project_Spt.Transcriptomics/29ntw7sh4r-2/data")
TILE_PATH = BASE_PATH / "tiles"
TILE_PATH.mkdir(parents=True, exist_ok=True)


meta = pd.read_csv(BASE_PATH / "metadata.csv")
meta = meta.iloc[:21,:]

adata_list = []
for index,row in meta.iterrows():
    cm_path = BASE_PATH / row["stdata"].split(".gz")[0]
    spot_path = BASE_PATH / row["spots"].split(".gz")[0]
    img_path = BASE_PATH / row["img"]
    Sample = row["pateint"] +"_"+ row["replicate"]
    adata = st.read.file_table(cm_path)
    spot_df = pd.read_csv(spot_path,index_col=0)
    comm_index = pd.Series(list(set(spot_df.index).intersection(set(adata.obs_names))))
    adata = adata[comm_index]
    adata.obs["imagecol"] = spot_df["X"]
    adata.obs["imagerow"] = spot_df["Y"]
    st.add.image(adata, img_path, library_id=Sample)
    adata.obs["type"] = row["type"]
    
    st.pp.filter_genes(adata,min_cells=3)
    sc.pp.filter_cells(adata,min_counts=100)
    st.pp.log1p(adata)
    
    tiling(adata, out_path=TILE_PATH, crop_size=299)
    adata_list.append(adata)

adata_all = adata_list[0].concatenate(
    adata_list[1:],
    batch_key="library_id",
    uns_merge="unique",
    batch_categories=[list(d.keys())[0]
                      for d in[adata_list[i].uns["spatial"] 
                      for i in range(len(adata_list))
                              ]
                     ],
)

Trying to set attribute `.obs` of view, copying.


Added tissue image to the object!
Log transformation step is finished in adata.X


Tiling image: 100%|██████████ [ time left: 00:00 ]
Trying to set attribute `.obs` of view, copying.
Tiling image:   0%|           [ time left: ? ]

Added tissue image to the object!
Log transformation step is finished in adata.X


Tiling image: 100%|██████████ [ time left: 00:00 ]
Trying to set attribute `.obs` of view, copying.


...

Tiling image: 100%|██████████ [ time left: 00:00 ]
Trying to set attribute `.obs` of view, copying.


Added tissue image to the object!


Tiling image:   0%|           [ time left: ? ]

Log transformation step is finished in adata.X


Tiling image: 100%|██████████ [ time left: 00:00 ]


In [9]:
ems_df = pd.read_csv("D:/onkar/Projects/Project_Spt.Transcriptomics/29ntw7sh4r-2/data/ensembl.tsv", sep="\t")
a = adata_all.var_names.intersection(ems_df["Ensembl ID(supplied by Ensembl)"])
b = ems_df["Approved symbol"][ems_df["Ensembl ID(supplied by Ensembl)"].isin(a)]
var_dic = dict(zip(a,b))
adata_all = adata_all[:,a].copy()
adata_all.var_names = adata_all.var_names.map(var_dic)
adata_all.to_df()

,A1BG,A2M,A4GALT,AAAS,AACS,AAGAB,AAK1,AAMDC,AAMP,AAR2,...,ZSWIM8,ZW10,ZWILCH,ZWINT,ZXDB,ZXDC,ZYG11B,ZYX,ZZEF1,ZZZ3
9x26-BC23209_C1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
15x13-BC23209_C1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,1.098612,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
6x22-BC23209_C1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,1.098612,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
10x14-BC23209_C1,0.0,0.693147,0.0,0.0,0.0,0.0,0.0,0.693147,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,1.098612,0.0,0.693147,0.0
19x12-BC23209_C1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26x21-BC23269_D1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.693147,0.0,0.000000,0.0
8x23-BC23269_D1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,1.098612,0.0,0.693147,0.0
22x23-BC23269_D1,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0
12x16-BC23269_D1,0.0,0.693147,0.0,0.0,0.0,0.0,0.0,0.693147,0.693147,0.000000,...,0.000000,0.0,0.0,0.0,0.0,0.0,1.098612,0.0,0.693147,0.0


# ST-Net Test Set

In [4]:
BASE_PATH_test = Path("D:/onkar/Projects/Project_Spt.Transcriptomics/29ntw7sh4r-2/data")
TILE_PATH_test = BASE_PATH_test / "tiles_test"
TILE_PATH_test.mkdir(parents=True, exist_ok=True)


meta_test = pd.read_csv(BASE_PATH_test / "metadata.csv")
meta_test = meta_test.iloc[21:27,:]

adata_list_test = []
for index,row in meta_test.iterrows():
    cm_path_test = BASE_PATH_test / row["stdata"].split(".gz")[0]
    spot_path_test = BASE_PATH_test / row["spots"].split(".gz")[0]
    img_path_test = BASE_PATH_test / row["img"]
    Sample_test = row["pateint"] +"_"+ row["replicate"]
    adata_test = st.read.file_table(cm_path_test)
    spot_df_test = pd.read_csv(spot_path_test,index_col=0)
    comm_index_test = pd.Series(list(set(spot_df_test.index).intersection(set(adata_test.obs_names))))
    adata_test = adata_test[comm_index_test]
    adata_test.obs["imagecol"] = spot_df_test["X"]
    adata_test.obs["imagerow"] = spot_df_test["Y"]
    st.add.image(adata_test, img_path_test, library_id=Sample_test)
    adata_test.obs["type"] = row["type"]
    
    st.pp.filter_genes(adata_test,min_cells=3)
    sc.pp.filter_cells(adata_test,min_counts=100)
    st.pp.log1p(adata_test)
    
    tiling(adata_test, out_path=TILE_PATH_test, crop_size=299)
    adata_list_test.append(adata_test)

adata_all_test = adata_list_test[0].concatenate(
    adata_list_test[1:],
    batch_key="library_id",
    uns_merge="unique",
    batch_categories=[list(d.keys())[0]
                      for d in[adata_list_test[i].uns["spatial"] 
                      for i in range(len(adata_list_test))
                              ]
                     ],
)

Trying to set attribute `.obs` of view, copying.


Added tissue image to the object!
Log transformation step is finished in adata.X


Tiling image: 100%|██████████ [ time left: 00:00 ]
Trying to set attribute `.obs` of view, copying.


Added tissue image to the object!
Log transformation step is finished in adata.X


Tiling image: 100%|██████████ [ time left: 00:00 ]
Trying to set attribute `.obs` of view, copying.


...

Tiling image: 100%|██████████ [ time left: 00:00 ]
Trying to set attribute `.obs` of view, copying.


Added tissue image to the object!
Log transformation step is finished in adata.X


Tiling image: 100%|██████████ [ time left: 00:00 ]
Trying to set attribute `.obs` of view, copying.


Added tissue image to the object!
Log transformation step is finished in adata.X


Tiling image: 100%|██████████ [ time left: 00:00 ]


In [7]:
ems_df_test = pd.read_csv("D:/onkar/Projects/Project_Spt.Transcriptomics/29ntw7sh4r-2/data/ensembl.tsv", sep="\t")
a_test = adata_all_test.var_names.intersection(ems_df_test["Ensembl ID(supplied by Ensembl)"])
b_test = ems_df_test["Approved symbol"][ems_df_test["Ensembl ID(supplied by Ensembl)"].isin(a)]
var_dic_test = dict(zip(a_test,b_test))
adata_all_test = adata_all_test[:,a_test].copy()
adata_all_test.var_names = adata_all_test.var_names.map(var_dic_test)
adata_all_test.to_df()

Variable names are not unique. To make them unique, call `.var_names_make_unique`.


,A1BG,A2M,A4GALT,AAAS,AACS,AAGAB,AAK1,AAMDC,AAMP,AAR2,...,ZW10,ZWILCH,ZWINT,ZXDB,ZXDC,ZYG11A,ZYG11B,ZYX,ZZEF1,ZZZ3
15x13-BC23272_D2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
28x26-BC23272_D2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.693147,0.000000,0.0,0.0,0.000000,0.0
10x14-BC23272_D2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.693147,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
10x9-BC23272_D2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,1.386294,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
3x21-BC23272_D2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.693147,0.000000,0.000000,0.0,0.0,1.098612,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26x21-BC23277_E2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.693147,0.0
8x23-BC23277_E2,0.0,0.000000,0.693147,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,1.098612,0.000000,0.0,0.0,0.000000,0.0
22x23-BC23277_E2,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,1.098612,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
12x16-BC23277_E2,0.0,1.098612,0.693147,0.0,0.0,0.0,1.386294,0.000000,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.693147,0.693147,0.0,0.0,0.000000,0.0


# ResNet50 Features

## ST-Net

### Train

In [10]:
import os; import pandas as pd; import numpy as np
import PIL; from PIL import Image
PIL.Image.MAX_IMAGE_PIXELS = 933120000
import os; import glob
from PIL import Image; import matplotlib.pyplot as plt; 
from keras.utils import np_utils
from keras.models import Sequential
from keras.applications import VGG16, ResNet50, inception_v3, DenseNet121
from keras.applications import imagenet_utils
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import Dense, Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D
import warnings

wd = "D:/onkar/Projects/Project_Spt.Transcriptomics/Output_files"

def ResNet50_features_train(test, pre_model):
    
    x_scratch_test = []
    for imagePath in test:
        image = load_img(imagePath, target_size=(299,299))
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)
        x_scratch_test.append(image)

    x_test = np.vstack(x_scratch_test)
    features_test = pre_model.predict(x_test, batch_size=32)
    features_flatten_test = features_test.reshape((features_test.shape[0], 2048))
    features_flatten_test = pd.DataFrame(features_flatten_test)
    features_flatten_test.index = adata_all.obsm.to_df().index
    adata_all.obsm["Resnet50_Train_Features"] = features_flatten_test
    
train = adata_all.obs["tile_path"]
model = ResNet50(weights="imagenet", include_top=False, input_shape=(299,299, 3), pooling="avg")
ResNet50_features_train(train, model)

### Test

In [12]:
wd = "D:/onkar/Projects/Project_Spt.Transcriptomics/Output_files"

def ResNet50_features_test(test, pre_model):
    
    x_scratch_test = []
    for imagePath in test:
        image = load_img(imagePath, target_size=(299,299))
        image = img_to_array(image)
        image = np.expand_dims(image, axis=0)
        x_scratch_test.append(image)

    x_test = np.vstack(x_scratch_test)
    features_test = pre_model.predict(x_test, batch_size=32)
    features_flatten_test = features_test.reshape((features_test.shape[0], 2048))
    features_flatten_test = pd.DataFrame(features_flatten_test)
    features_flatten_test.index = adata_all_test.obsm.to_df().index
    adata_all_test.obsm["Resnet50_Test_Features"] = features_flatten_test
    
#train = Sample1.obs["tile_path"]
test = adata_all_test.obs["tile_path"]
ResNet50_features_test(test, model)

In [13]:
adata_all.obsm["Resnet50_Train_Features"].to_csv(wd+'/Filtered_ST-Net_train.csv')
adata_all_test.obsm["Resnet50_Test_Features"].to_csv(wd+'/Filtered_ST-Net_test.csv')

## ST-Net

In [14]:
wd = "D:/onkar/Projects/Project_Spt.Transcriptomics/Output_files/"
biomarker_list_ST = ['COX6C', "MALAT1", 'TTLL12', "PABPC1", "GNAS", "HSP90AB1", "ATP1A1", "B2M", "FASN", "ACTG1", "HLA-B"]


X_ST = adata_all.obsm["Resnet50_Train_Features"]#pd.read_csv('D:/onkar/Projects/Project_Spt.Transcriptomics/Output_files/Resnet_unnorm_train.csv')#
#X_ST = X_ST.iloc[:,1:]
test_X_ST = adata_all_test.obsm["Resnet50_Test_Features"]#Sample3.obsm["Resnet50_Test_Features"]#pd.read_csv('D:/onkar/Projects/Project_Spt.Transcriptomics/Output_files/Filtered_ST-Net.csv')#
#test_X_ST = test_X_ST.iloc[:,1:]

Y_ST = adata_all.to_df()#[adata_all.to_df().sum().sort_values(ascending=False).index[:10]] 
test_Y_ST = adata_all_test.to_df()#[adata_all.to_df().sum().sort_values(ascending=False).index[:10]] 
number_of_top_genes = 10
model = lgb.LGBMClassifier()#MLPClassifier(alpha=1, max_iter=7500)#

def multiclass_roc_auc_score(truth, pred, average="macro"):
    lb = LabelBinarizer()
    lb.fit(truth)
    truth = lb.transform(truth)
    pred = lb.transform(pred)
    return roc_auc_score(truth, pred, average=average)

Y_ST = Y_ST[biomarker_list_ST]#.iloc[:,:number_of_top_genes]#
genes = Y_ST.columns
MinMax_scaler_y_ST = preprocessing.MinMaxScaler(feature_range =(0, 1))
Y_ST = MinMax_scaler_y_ST.fit_transform(Y_ST) 
Y_ST = pd.DataFrame(data=Y_ST)
Y_ST = Y_ST.apply(lambda x: pd.cut(x, bins=[-0.01, 0.5, 1.05], labels = [0, 1]))
#Y=Y.apply(lambda x: pd.qcut(x, 3,duplicates='drop',labels=False))

test_Y_ST=test_Y_ST[biomarker_list_ST]#.iloc[:,:number_of_top_genes]#
test_Y_ST=MinMax_scaler_y_ST.fit_transform(test_Y_ST)
test_Y_ST=pd.DataFrame(data=test_Y_ST)
test_Y_ST = test_Y_ST.apply(lambda x: pd.cut(x, bins=[-0.01, 0.5, 1.05], labels = [0, 1]))
#test_Y=test_Y.apply(lambda x: pd.qcut(x, 3,duplicates='drop',labels=False))

Y_ST = Y_ST.astype('int64')
test_Y_ST = test_Y_ST.astype('int64')

In [15]:
clf_2 = MultiOutputClassifier(model).fit(X_ST, Y_ST)

In [16]:
y_pred_test_1_ST = clf_2.predict_proba(test_X_ST)
y_pred_test_1_ST = np.array(y_pred_test_1_ST)

y_pred_test_2_ST = clf_2.predict(test_X_ST)
y_pred_test_2_ST = pd.DataFrame(y_pred_test_2_ST)

In [17]:
auroc_ST = []
for col in test_Y_ST:
    score_multi_ST = roc_auc_score(test_Y_ST[col], y_pred_test_2_ST[col])#, average='weighted'
    auroc_ST.append(score_multi_ST)
    
result_ST = pd.DataFrame()
result_ST["auroc"] = auroc_ST
result_ST.index = genes

In [18]:
result_ST.sort_values("auroc", ascending=False)

,auroc
COX6C,0.5
MALAT1,0.5
TTLL12,0.5
PABPC1,0.5
GNAS,0.5
HSP90AB1,0.5
ATP1A1,0.5
B2M,0.5
FASN,0.5
ACTG1,0.5


In [ ]:
from sklearn.metrics import confusion_matrix as cm
cm(test_Y_ST[1],y_pred_test_2_ST[1])